<a href="https://colab.research.google.com/github/sugarforever/LangChain-Tutorials/blob/main/LangChain_Pinecone_Serverless.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain-community==0.0.13 pinecone-client==3.0.0 openai -q -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.9/199.9 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.3/222.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.6/218.6 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is 

In [9]:
from google.colab import userdata

PINECONE_INDEX_NAME = 'serverless-example'

import os

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')

In [7]:
from langchain_community.chat_models import ChatOpenAI
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Pinecone
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from pinecone import Pinecone as PineconeClient

In [10]:
vectorstore = Pinecone.from_existing_index(index_name=PINECONE_INDEX_NAME, embedding=OpenAIEmbeddings())

In [12]:
!wget -O Optimism_and_StarkNet.pdf https://arxiv.org/pdf/2210.16610.pdf

--2024-01-17 19:49:34--  https://arxiv.org/pdf/2210.16610.pdf
Resolving arxiv.org (arxiv.org)... 151.101.67.42, 151.101.3.42, 151.101.195.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.67.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 505763 (494K) [application/pdf]
Saving to: ‘Optimism_and_StarkNet.pdf’

Optimism_and_StarkN 100%[===================>] 493.91K  --.-KB/s    in 0.04s   

2024-01-17 19:49:35 (11.1 MB/s) - ‘Optimism_and_StarkNet.pdf’ saved [505763/505763]



In [19]:
!pip install -q -U langchain pypdf tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [15]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Optimism_and_StarkNet.pdf")
documents = loader.load()

In [17]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)
split_docs = text_splitter.split_documents(documents)

In [20]:
vectorstore.add_documents(split_docs)

['f5e75a8e-324c-43ea-b4da-5ab514fd96d0',
 '22747fb7-d14c-46c6-ba93-8ce35f21dddf',
 'c1631bf3-4950-4c80-942b-406172b68f62',
 'd15d89e6-bf52-4d23-b50b-2a5f96975693',
 'eb1b7a14-51a0-4f9d-a3ff-dda95baae02c',
 '521b82dc-0ba9-47d8-9e5f-175313640348',
 '9309945e-a415-4e74-ae00-034b63d6d639',
 '41cba134-ed92-476d-a1a8-64584f15edef',
 '558fdcb4-a891-4896-8b13-7c21089f1a02',
 '8ec311af-dd62-4dd5-af9d-3bc064923285',
 'be4842d4-7405-4b3f-8a5c-81cf8bb434f2',
 '402317c3-b14d-45db-9c0b-591970d59ebd',
 '5bb0366a-9df9-49fc-be6a-08df51c3fd5f',
 'cee2c5bc-406d-4020-addc-76cba22657cc',
 'ff5be6ea-916d-4149-a5bf-70723ec9c3f2',
 '3e4b2181-508e-4b12-80cf-4470bdddc1fa',
 'c970cc8a-65b3-4b54-8ac8-2dae2d5a73d2',
 '7c4d6831-e20a-408d-a87c-734bb7130dd0',
 '21aaf5dd-0335-4516-807c-5f747ac4a22b',
 'a1e7d351-c391-4602-b0ce-94b7050079be',
 '1e6a2dd4-3ced-4a47-a633-19fc4b8294d9',
 '53d9f9f9-d768-4a6f-a279-f56b52a4896d',
 '5bff99ee-241d-4fca-87b0-c17360f156de',
 '23e7e451-bb06-4c4a-b445-8974bd135455',
 '23de79e9-8763-

In [21]:
retriever = vectorstore.as_retriever()

In [23]:
template = """Answer the question based only on the following context:
{context}
Question: {question}
"""

chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | ChatPromptTemplate.from_template(template)
    | ChatOpenAI(temperature=0, model="gpt-3.5-turbo-16k")
    | StrOutputParser()
)

In [27]:
chain.invoke("What is optimisitic rollup?")

'Optimistic Rollup is a scalability solution for decentralized blockchains that locks the most value. It is based on the Ethereum Virtual Machine (EVM) and allows for strong development due to compatibility with existing Ethereum tools and the use of Solidity. It is a type of Rollup that accepts the output of blocks optimistically without verifying their execution.'

In [29]:
chain.invoke("What is the conclusion?")

'The conclusion is that rollups are the most promising solution available today to solve the scalability problem in decentralized blockchains, ushering in the era of modular blockchains as opposed to monolithic blockchains.'